# PCA 

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io  
from sklearn.decomposition import PCA
import glob 
import time

total_time = 0.0
sum_fit = 0.0
sum_transform = 0.0
start_total = time.process_time()
i = 0

os.mkdir('PCA') 
path = "Resized_sample/*.jpeg*"
for image in glob.glob(path):
    img = io.imread(image)
    img = img.astype(np.uint8)
#    print(img.shape)
#    plt.imshow(img, cmap="gray") 
    
    start_fit = time.process_time()
    fit_pca = PCA(n_components=50).fit(img)  # approx 98.143%(1), 96.61%(2) information in an image [PC1 greater, more info]
    sum_fit += time.process_time() - start_fit
    
    start_transform = time.process_time()
    transformed = fit_pca.transform(img)
    sum_transform += time.process_time() - start_transform
    
    projected = fit_pca.inverse_transform(transformed)
    transformed = transformed.astype(np.uint8)
#     print(transformed.shape)
#     plt.imshow(projected, cmap="gray")
    io.imsave("PCA/image%05i.jpeg" %i, transformed)
    i += 1 
    
total_time += time.process_time() - start_total

avg_fit = sum_fit/100;
avg_transform = sum_transform/100

print("Total Fit Time: ", sum_fit)
print("Average Fit time: ", avg_fit)
print("Total Transform Time: ", sum_transform)
print("Average Transform time: ", avg_transform)
print("Total Time required for execution: ", total_time)


Total Fit Time:  12.625
Average Fit time:  0.12625
Total Transform Time:  0.0625
Average Transform time:  0.000625
Total Time required for execution:  13.703125
